> Baseline model

## BASE MODEL - Logistic Regression


In [135]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, classification_report
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

### Loading Data

In [136]:
data = pd.read_csv('../data/processed/data_abnormal_values_treated_scaled.csv') 
data.head()

,Unnamed: 0,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,0,-1.356073,M,ATA,0.446245,0.876016,0,Normal,1.296470,N,-0.862606,Up,0
1,1,-0.408656,F,NAP,1.667669,-1.194233,0,Normal,0.643613,N,0.107330,Flat,1
2,2,-1.671879,M,ATA,-0.164466,0.762057,0,ST,-1.722993,N,-0.862606,Up,0
3,3,-0.513925,F,ASY,0.324103,-0.548467,0,Normal,-1.314958,Y,0.592298,Flat,1
4,4,0.117686,M,NAP,1.056957,-0.909337,0,Normal,-0.743708,N,-0.862606,Up,0


## Seperate X and Y

In [137]:
# seperate X and y dataframes

feature_columns = data.columns[1:-1]

X = data[feature_columns]
y = data['HeartDisease'] 

In [138]:
y.value_counts()

HeartDisease
0    390
1    356
Name: count, dtype: int64


## Feature Engineering
Label Encoding

Use LabelEncoder to assign an integer to each category 

Categorical features: Sex, ChestPainType, FastingBS, RestingECG, ExerciseAngina, ST_Slope


In [139]:
# define categorical columns

cat_cols = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']

# Using OrdinalEncoder 
categorical_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder())
])

# ColumnTransformer
preproc = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_cols),
    ],
    remainder='passthrough'  # Leave the rest of the columns unchanged
)
preproc

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 Pipeline(steps=[('encoder',
                                                  OrdinalEncoder())]),
                                 ['Sex', 'ChestPainType', 'FastingBS',
                                  'RestingECG', 'ExerciseAngina',
                                  'ST_Slope'])])

In [140]:
# Apply the transformation
X_transformed = preproc.fit_transform(X)

# Reconstruct a DataFrame
new_columns = cat_cols + [col for col in X.columns if col not in cat_cols]
X_transformed_df = pd.DataFrame(X_transformed, columns=new_columns)

X_transformed_df.head()

,Sex,ChestPainType,FastingBS,RestingECG,ExerciseAngina,ST_Slope,Age,RestingBP,Cholesterol,MaxHR,Oldpeak
0,1.0,1.0,0.0,1.0,0.0,2.0,-1.356073,0.446245,0.876016,1.296470,-0.862606
1,0.0,2.0,0.0,1.0,0.0,1.0,-0.408656,1.667669,-1.194233,0.643613,0.107330
2,1.0,1.0,0.0,2.0,0.0,2.0,-1.671879,-0.164466,0.762057,-1.722993,-0.862606
3,0.0,0.0,0.0,1.0,1.0,1.0,-0.513925,0.324103,-0.548467,-1.314958,0.592298
4,1.0,2.0,0.0,1.0,0.0,2.0,0.117686,1.056957,-0.909337,-0.743708,-0.862606


## Pipeline

In [141]:
# 4. Re-define the Model Pipeline
model_pipeline = Pipeline([
    ('Feature Engineering', preproc),
    ('classifier', LogisticRegression(random_state=42)) 
])

model_pipeline

c:\Users\israi\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('Feature Engineering',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['Sex', 'ChestPainType',
                                                   'FastingBS', 'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', LogisticRegression(random_state=42))])

## Train / Test Split

In [142]:
# train/test splitting

X_train, X_test, y_train, y_test = train_test_split(
    X, y, # dataset
    train_size = 0.8, # the size of training set = 80%
    shuffle = True, #to avoid ordering effect
    stratify = y, # maintain the distribution of y classes in both training and test sets
    random_state = 42
)

## Cross Validation

In [143]:

# Define the scoring metrics
scoring = [
    'accuracy',
    'precision',
    'recall',
    'f1',
    'roc_auc'
]

result_dict = cross_validate(model_pipeline, X_train, y_train, cv=5, scoring=scoring)

result = pd.DataFrame(result_dict)
print("\nCross-Validation Results:")
print(result)


Cross-Validation Results:
   fit_time  score_time  test_accuracy  test_precision  test_recall   test_f1  \
0  0.017902    0.015970       0.866667        0.859649     0.859649  0.859649   
1  0.023202    0.008086       0.857143        0.819672     0.892857  0.854701   
2  0.000000    0.018932       0.848739        0.842105     0.842105  0.842105   
3  0.020816    0.013061       0.806723        0.803571     0.789474  0.796460   
4  0.017961    0.014003       0.865546        0.847458     0.877193  0.862069   

   test_roc_auc  
0      0.941799  
1      0.903628  
2      0.920770  
3      0.867006  
4      0.937182  


## Model Performance

In [144]:
model_pipeline.fit(X_train, y_train)
#  Evaluate Baseline Model 
y_pred = model_pipeline.predict(X_test)
y_proba = model_pipeline.predict_proba(X_test)[:, 1] # Probability for the positive class (1)

# Calculate key metrics
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)

print("\n" + "="*60)
print("BASELINE MODEL PERFORMANCE (Logistic Regression)")
print("="*60)
print(f"Accuracy:        {accuracy:.4f}")
print(f"AUC Score:       {auc:.4f}")
print(f"F1 Score:        {f1:.4f}")
print("-" * 60)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("-" * 60)




BASELINE MODEL PERFORMANCE (Logistic Regression)
Accuracy:        0.8667
AUC Score:       0.9218
F1 Score:        0.8592
------------------------------------------------------------

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.88      0.87        78
           1       0.87      0.85      0.86        72

    accuracy                           0.87       150
   macro avg       0.87      0.87      0.87       150
weighted avg       0.87      0.87      0.87       150

------------------------------------------------------------


## Confusion Matrix

## ROC Curve